In [2]:
import random
import numpy as np

In [ ]:
class MundoWumpus:
    def __init__(self, n=4, n_abismos=1):
        self.n = n
        self.n_abismos = n_abismos
        self.pos_inicial = (0, 0)
        self._posicionar_elementos()
        self.reset()

    def _posicionar_elementos(self):
        posicoes_disponiveis = [(i, j) for i in range(self.n) for j in range(self.n)]
        posicoes_disponiveis.remove(self.pos_inicial)

        self.abismos = []
        for _ in range(self.n_abismos):
            if not posicoes_disponiveis: break
            pos_abismo = random.choice(posicoes_disponiveis)
            self.abismos.append(pos_abismo)
            posicoes_disponiveis.remove(pos_abismo)

        if not posicoes_disponiveis: self.pos_wumpus = None
        else:
            self.pos_wumpus = random.choice(posicoes_disponiveis)
            posicoes_disponiveis.remove(self.pos_wumpus)

        if not posicoes_disponiveis: self.pos_ouro = None
        else: self.pos_ouro = random.choice(posicoes_disponiveis)
    
    def get_estado_agente(self):
        return (self.agente_pos[0], self.agente_pos[1], self.agente_tem_flecha, self.wumpus_vivo)

    def reset(self):
        self.agente_pos = self.pos_inicial
        self.wumpus_vivo = True
        self.agente_tem_flecha = True
        self.ouro_coletado = False
        return self.get_estado_agente()

    def get_percepcoes(self, pos):
        percepcoes = []
        px, py = pos
        for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]:
            if (px + dx, py + dy) in self.abismos:
                percepcoes.append("Brisa")
                break
        if self.wumpus_vivo:
             for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]:
                if (px + dx, py + dy) == self.pos_wumpus:
                    percepcoes.append("Fedor")
                    break
        if not self.ouro_coletado and pos == self.pos_ouro:
            percepcoes.append("Brilho")
        return percepcoes

    def step(self, action):
        recompensa = -1  # Custo padrão de movimento
        terminou = False
        x, y = self.agente_pos

        pos_anterior = self.agente_pos

        if action == 'n': self.agente_pos = (max(0, x - 1), y)
        elif action == 's': self.agente_pos = (min(self.n - 1, x + 1), y)
        elif action == 'o': self.agente_pos = (x, max(0, y - 1))
        elif action == 'l': self.agente_pos = (x, min(self.n - 1, y + 1))
        
        if self.agente_pos == pos_anterior and action in ['n','s','l','o']:
            recompensa = -5

        elif action == 'pegar':
            if self.agente_pos == self.pos_ouro and not self.ouro_coletado:
                recompensa = 1000
                self.ouro_coletado = True
                terminou = True  
            else:
                recompensa = -10

        elif action.startswith('atirar_'):
            if self.agente_tem_flecha:
                self.agente_tem_flecha = False
                recompensa = -10  # Custo por usar a flecha
                
                direcoes = {'atirar_n': (-1, 0), 'atirar_s': (1, 0), 'atirar_o': (0, -1), 'atirar_l': (0, 1)}
                dx, dy = direcoes[action]
                
                ax, ay = self.agente_pos
                wx, wy = self.pos_wumpus
                if self.wumpus_vivo and ((dx != 0 and ax == wx and (wy - ay) * dy > 0) or \
                                         (dy != 0 and ay == wy and (wx - ax) * dx > 0)):
                    recompensa += 100
                    self.wumpus_vivo = False
            else:
                recompensa = -20 

        if self.agente_pos in self.abismos or \
           (self.agente_pos == self.pos_wumpus and self.wumpus_vivo):
            recompensa = -1000
            terminou = True

        return self.get_estado_agente(), recompensa, terminou

In [4]:
class AgenteQLearning:
    def __init__(self, actions, alpha=0.1, gamma=0.9, epsilon=1.0):
        self.q_table = {}
        self.actions = actions
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon #Pondera a probabilidade dele escolher a melhor ação ou ação aleatória

    def get_q_value(self, state, action):
        if state not in self.q_table:
            self.q_table[state] = {a: 0.0 for a in self.actions}
        return self.q_table[state][action]

    def escolher_acao(self, state):
        if random.random() < self.epsilon:
            return random.choice(self.actions)
        else:
            q_values = [self.get_q_value(state, a) for a in self.actions]
            max_q = max(q_values)
            best_actions = [self.actions[i] for i, q in enumerate(q_values) if q == max_q]
            return random.choice(best_actions)

    def aprender(self, state, action, reward, next_state):
        old_q = self.get_q_value(state, action)
        future_q_values = [self.get_q_value(next_state, a) for a in self.actions]
        max_future_q = max(future_q_values)
        new_q = old_q + self.alpha * (reward + self.gamma * max_future_q - old_q)
        self.q_table[state][action] = new_q

In [17]:
def jogo_q_learning(episodios=10000, n_abismos=10):
    env = MundoWumpus(n=4, n_abismos=n_abismos)
    acoes = ['n', 's', 'l', 'o', 'pegar', 'atirar_n', 'atirar_s', 'atirar_l', 'atirar_o']
    agente = AgenteQLearning(actions=acoes)
    
    epsilon_decay = 0.99
    min_epsilon = 0.01

    print("Fase de treinamento iniciada")
    for episodio in range(episodios):
        state = env.reset()
        terminou = False
        passos = 0
        
        while not terminou and passos < 100:
            action = agente.escolher_acao(state)
            next_state, reward, terminou = env.step(action)
            agente.aprender(state, action, reward, next_state)
            state = next_state
            passos += 1
        
        if agente.epsilon > min_epsilon:
            agente.epsilon *= epsilon_decay

    
    print("\nFase de treinamento concluida")

    print("\nExecutando melhor jogo encontrado")
    agente.epsilon = 0
    state = env.reset()
    terminou = False
    caminho = [state[0:2]]
    acoes_executadas = []

    print(f"Configuração do Mundo: Wumpus em {env.pos_wumpus}, Ouro em {env.pos_ouro}, Abismos em {env.abismos}")
    
    passos = 0
    while not terminou and passos < 50:
        posicao_atual = state[0:2]
        print(f"Estado: {state} | Posição: {posicao_atual} | Percepções: {env.get_percepcoes(posicao_atual)}")
        
        action = agente.escolher_acao(state)
        acoes_executadas.append(action)
        
        next_state, _, terminou = env.step(action)
        state = next_state
        caminho.append(state[0:2])
        passos += 1

    print(f"\nÚltimo estado: {state} | Posição Final: {state[0:2]}")
    print(f"Caminho percorrido (posições): {caminho}")
    print(f"Ações executadas: {acoes_executadas}")

    if env.ouro_coletado:
        print("\nResultado: Sucesso! O agente coletou o ouro.")
    else:
        print("\nResultado: Falha. O agente não completou o objetivo.")

In [18]:
jogo_q_learning()

Fase de treinamento iniciada

Fase de treinamento concluida

Executando melhor jogo encontrado
Configuração do Mundo: Wumpus em (1, 2), Ouro em (0, 3), Abismos em [(1, 3), (3, 1), (0, 1), (3, 2), (3, 3), (2, 2), (2, 1), (3, 0), (0, 2), (2, 0)]
Estado: (0, 0, True, True) | Posição: (0, 0) | Percepções: ['Brisa']
Estado: (1, 0, True, True) | Posição: (1, 0) | Percepções: ['Brisa']
Estado: (1, 1, True, True) | Posição: (1, 1) | Percepções: ['Brisa', 'Fedor']
Estado: (1, 0, True, True) | Posição: (1, 0) | Percepções: ['Brisa']
Estado: (0, 0, True, True) | Posição: (0, 0) | Percepções: ['Brisa']
Estado: (1, 0, True, True) | Posição: (1, 0) | Percepções: ['Brisa']
Estado: (1, 1, True, True) | Posição: (1, 1) | Percepções: ['Brisa', 'Fedor']
Estado: (1, 0, True, True) | Posição: (1, 0) | Percepções: ['Brisa']
Estado: (1, 1, True, True) | Posição: (1, 1) | Percepções: ['Brisa', 'Fedor']
Estado: (1, 0, True, True) | Posição: (1, 0) | Percepções: ['Brisa']
Estado: (0, 0, True, True) | Posição: (

Fontes: Slides da aula, google gemini